In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

# load data

test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
samples = pd.read_csv("sample_submission.csv") # how data we send to Kaggle should look like

# train data consists out of 5 columns : "id", "keyword", "location", "text", "target", while test data has no "target"
# as we do not focus on reaching very high accuracy I'll delete "keyword" and "location"
# i will also delete "id" as we can index by row number

train_data.drop(columns = ["keyword", "id", "location"], inplace = True)
test_data.drop(columns = ["keyword", "id", "location"], inplace = True)

train_short = pd.DataFrame(train_data.values[:10,:])

# train data consist out of 7613 observations, including 4342 zeroes (false alarm) and 3271 ones (boooom)
# test data consist out of 3261 observation

# for simplicity let's reorganize it into arrays

train_text = train_data["text"]
print(train_text.shape[0])
train_target = train_data["target"]
test_text = test_data["text"]

# tokenize input to make it understandable for Bertie

model_name = 'bert-base-uncased' # smaller version of Bert NOT seeing any difference between small and big letters (B = b) or special signs (ó = o)

tokenizer = BertTokenizer.from_pretrained(model_name)

train_text_toc = []
test_text_toc = []
'''
for sentence in train_text:
    train_text_toc.append(tokenizer(sentence, 
                                    padding = "max_length", # if we want to pad some "zeroes" to make all sentences of equal length
                                    max_length = 54,        # maximum length (in words) of sentence in train_data (I counted spaces)
                                    truncation = True,      # if some word is actually longer, than it shall be cut
                                    return_tensors = "pt")) # we want to get pytorch tensors
for sentence in test_text:
    test_text_toc.append(tokenizer(sentence, 
                                    padding = "max_length",
                                    max_length = 54,      
                                    truncation = True,     
                                    return_tensors = "pt"))
    
# NOTE: a tokenized sentenced consist out of 3 lists of tensors:
# 1. tokenized words + some "zeroes" to make the length constant
# 2. sentence number - we have here mostly just zeroes as we have usually just one sentence
# 3. info if a word is a real word or just padded "zero"
    
# and now load train data to a DataLoader

batch_size = 100

data = DataLoader(TensorDataset(train_text_toc, train_target), shuffle = True, batch_size = batch_size)
'''

input = tokenizer(train_text[0], 
                                    padding = "max_length", # if we want to pad some "zeroes" to make all sentences of equal length
                                    max_length = 54,        # maximum length (in words) of sentence in train_data (I counted spaces)
                                    truncation = True,      # if some word is actually longer, than it shall be cut
                                    return_tensors = "pt") # we want to get pytorch tensors

model = BertModel.from_pretrained(model_name)
output = model(**input)

(10, 2)
7613


KeyboardInterrupt: 

In [3]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1258,  0.4258,  0.1974,  ..., -0.1256,  0.2480, -0.0108],
         [ 0.7031,  0.8352, -0.0143,  ..., -0.1379,  1.1167, -0.3396],
         [ 1.0234,  0.1433,  0.4797,  ..., -0.2408,  0.1012, -0.2401],
         ...,
         [-0.1814,  0.3692,  0.5908,  ..., -0.0976, -0.1043, -0.2488],
         [-0.1037,  0.3431,  0.5569,  ..., -0.0352, -0.0170, -0.3094],
         [-0.0573,  0.3523,  0.5949,  ..., -0.0129,  0.0647, -0.4264]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.2255e-01, -4.7206e-01, -5.7987e-01,  6.0134e-01,  1.1315e-03,
         -3.3307e-02,  6.2188e-01,  2.1917e-01, -2.9744e-01, -9.9993e-01,
          1.2768e-01,  7.2085e-01,  9.8715e-01,  2.3808e-01,  8.7115e-01,
         -1.1834e-01,  1.8368e-01, -4.6653e-01,  3.2464e-01,  7.9548e-02,
          6.6343e-01,  9.9941e-01,  5.0706e-01,  2.3420e-01,  4.5916e-01,
          9.0418e-01, -5.7565e-01,  8.8735e-01,  9.5978e-01,  5.946